In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

d:\Programy\anaconda3\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'D:\Programy\anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [49]:
# !nvidia-smi

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cpu device


In [45]:
from typing import Optional, Callable, Tuple, Any
import numpy as np
from PIL import Image
import os


class MagicDataset(torch.utils.data.Dataset):
    def __init__(
        self, 
        img_dir: str,
        labels_file: Optional[str] = None,
        transform: Optional[Callable] = None,
        target_transform: Optional[Callable] = None
    ) -> None:

        self.img_dir = img_dir
        self.labels_file = labels_file
        self.img_list = os.listdir(self.img_dir)
        self.img_labels = pd.read_csv(labels_file).loc[:, "category_id"]

        self.transform = transform
        self.target_transform = target_transform

    def __len__(self) -> int:
        return len(self.img_list)

    def __getitem__(self, index: int):
        img_path = os.path.join(self.img_dir, self.img_list[index])
        image = Image.open(img_path)
        label = self.img_labels[index]

        if self.transform:
            image = self.transform(image)
            
        if self.target_transform:
            label = self.target_transform(label)

        return image, label
    

In [46]:
IMG_DIR = 'data/reference_images_part1_bboxes'
LABELS_FILE = 'data/bboxes_data.csv'
BATCH_SIZE = 64

In [39]:
len(os.listdir('data/reference_images_part1_bboxes'))

144

In [37]:
len(os.listdir('data\images_part1_valid_bboxes'))

6591

In [31]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [47]:
train_data = MagicDataset(img_dir=IMG_DIR, labels_file=LABELS_FILE, transform=train_transforms)

In [48]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

In [21]:
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\mikol/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [01:20<00:00, 580kB/s] 


In [ ]:
model

In [ ]:
# LR = 1e-4
# EPOCHS = 25

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LR)

In [ ]:
def inferece(model, criterion):
    model.eval()

    running_loss = 0.0
    running_corrects = 0
    running_micro_f1 = 0.0
    running_macro_f1 = 0.0
    y_true_all = []
    y_pred_all = []



    with torch.no_grad():
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            probs, preds = torch.max(F.softmax(outputs, dim=1), 1)    

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data).item()
            running_micro_f1 = micro_f1_score(preds.cpu(), labels.cpu()).item()
            running_macro_f1 = macro_f1_score(preds.cpu(), labels.cpu()).item()
            y_pred_all.extend(preds.cpu().detach().numpy())
            y_true_all.extend(labels.cpu().detach().numpy())

    mean_loss = running_loss / dataset_sizes['val']
    mean_acc = running_corrects / dataset_sizes['val']
    mean_micro_f1 = micro_f1_score.compute().item()
    mean_macro_f1 = macro_f1_score.compute().item()
   
    print(f"Mean Loss: {mean_loss:.4f} Accuracy: {mean_acc:.4f} Micro F1: {mean_micro_f1:.4f} Macro F1: {mean_macro_f1:.4f}")
    return y_true_all, y_pred_all
